<a href="https://colab.research.google.com/github/ShubhFuladi06/ANN-With-Batch-Normalization/blob/main/Batch%20Normalization%20ANN%20(MNIST%20dataset).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("fivethirtyeight")
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [25]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [26]:
tf.random.set_seed(42)
np.random.seed(42)

LAYERS = [ tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(100, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(10, activation="softmax")]


model = tf.keras.models.Sequential(LAYERS)

In [27]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [28]:
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 784)               0         
                                                                 
 dense_9 (Dense)             (None, 300)               235500    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 300)               0         
                                                                 
 dense_10 (Dense)            (None, 100)               30100     
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-tr

In [29]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid), verbose=2)

Epoch 1/10
1719/1719 - 6s - loss: 1.2819 - accuracy: 0.6229 - val_loss: 0.8886 - val_accuracy: 0.7160 - 6s/epoch - 3ms/step
Epoch 2/10
1719/1719 - 8s - loss: 0.7955 - accuracy: 0.7362 - val_loss: 0.7130 - val_accuracy: 0.7656 - 8s/epoch - 5ms/step
Epoch 3/10
1719/1719 - 5s - loss: 0.6816 - accuracy: 0.7721 - val_loss: 0.6427 - val_accuracy: 0.7900 - 5s/epoch - 3ms/step
Epoch 4/10
1719/1719 - 5s - loss: 0.6217 - accuracy: 0.7944 - val_loss: 0.5900 - val_accuracy: 0.8066 - 5s/epoch - 3ms/step
Epoch 5/10
1719/1719 - 5s - loss: 0.5832 - accuracy: 0.8074 - val_loss: 0.5582 - val_accuracy: 0.8202 - 5s/epoch - 3ms/step
Epoch 6/10
1719/1719 - 5s - loss: 0.5553 - accuracy: 0.8157 - val_loss: 0.5350 - val_accuracy: 0.8238 - 5s/epoch - 3ms/step
Epoch 7/10
1719/1719 - 5s - loss: 0.5338 - accuracy: 0.8225 - val_loss: 0.5157 - val_accuracy: 0.8304 - 5s/epoch - 3ms/step
Epoch 8/10
1719/1719 - 5s - loss: 0.5172 - accuracy: 0.8273 - val_loss: 0.5079 - val_accuracy: 0.8282 - 5s/epoch - 3ms/step
Epoch 9/

# BN approach one

In [30]:
del model

In [31]:
LAYERS_BN = [
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation="softmax")
]

model = tf.keras.models.Sequential(LAYERS_BN)

In [32]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization_6 (Batc  (None, 784)              3136      
 hNormalization)                                                 
                                                                 
 dense_12 (Dense)            (None, 300)               235500    
                                                                 
 batch_normalization_7 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_13 (Dense)            (None, 100)               30100     
                                                                 
 batch_normalization_8 (Batc  (None, 100)             

In [33]:
784 * 4, 300 * 4, 100*4 # mean, variance, gamma and beta

(3136, 1200, 400)

In [34]:
sum([3136, 1200, 400])

4736

In [35]:
266610 + 4736

271346

In [36]:
4736/2 # out of 4 we only train gamma and beta, and mean, variance are calculated internally

2368.0

In [37]:
266610 + 4736/2


268978.0

In [38]:
bn1 = model.layers[1]

In [39]:
bn1

In [40]:
for variable in bn1.variables:
  print(variable.name, variable.trainable)

batch_normalization_6/gamma:0 True
batch_normalization_6/beta:0 True
batch_normalization_6/moving_mean:0 False
batch_normalization_6/moving_variance:0 False


In [41]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [42]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid), verbose=2)

Epoch 1/10
1719/1719 - 8s - loss: 0.8293 - accuracy: 0.7221 - val_loss: 0.5539 - val_accuracy: 0.8156 - 8s/epoch - 5ms/step
Epoch 2/10
1719/1719 - 7s - loss: 0.5703 - accuracy: 0.8037 - val_loss: 0.4792 - val_accuracy: 0.8378 - 7s/epoch - 4ms/step
Epoch 3/10
1719/1719 - 7s - loss: 0.5161 - accuracy: 0.8214 - val_loss: 0.4425 - val_accuracy: 0.8494 - 7s/epoch - 4ms/step
Epoch 4/10
1719/1719 - 7s - loss: 0.4789 - accuracy: 0.8313 - val_loss: 0.4213 - val_accuracy: 0.8564 - 7s/epoch - 4ms/step
Epoch 5/10
1719/1719 - 8s - loss: 0.4549 - accuracy: 0.8406 - val_loss: 0.4052 - val_accuracy: 0.8616 - 8s/epoch - 4ms/step
Epoch 6/10
1719/1719 - 7s - loss: 0.4386 - accuracy: 0.8441 - val_loss: 0.3932 - val_accuracy: 0.8632 - 7s/epoch - 4ms/step
Epoch 7/10
1719/1719 - 7s - loss: 0.4255 - accuracy: 0.8503 - val_loss: 0.3830 - val_accuracy: 0.8650 - 7s/epoch - 4ms/step
Epoch 8/10
1719/1719 - 7s - loss: 0.4124 - accuracy: 0.8539 - val_loss: 0.3761 - val_accuracy: 0.8670 - 7s/epoch - 4ms/step
Epoch 9/

# BN approach 2

In [43]:
del model

In [44]:
LAYERS_BN_BIAS_FALSE = [
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(100, use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(10, activation="softmax")
]

model = tf.keras.models.Sequential(LAYERS_BN_BIAS_FALSE)

In [45]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization_9 (Batc  (None, 784)              3136      
 hNormalization)                                                 
                                                                 
 dense_15 (Dense)            (None, 300)               235200    
                                                                 
 batch_normalization_10 (Bat  (None, 300)              1200      
 chNormalization)                                                
                                                                 
 activation_2 (Activation)   (None, 300)               0         
                                                                 
 dense_16 (Dense)            (None, 100)              